In [1]:
import os
import sys
import copy
pdir = os.path.dirname(os.getcwd())
sys.path.append(pdir)

import torch
import numpy as np

import utils
import wandb

model_dir = 'models'
data_dir = 'tree_points.pt'

In [2]:
trees_data = torch.load(data_dir)

In [3]:
sweep_config = {
  "name":"laser-trees-bayes",
  "method":"bayes",
  "metric":{
      "name":"Best_acc",
      "goal":"maximize"
  },
  "parameters":{
    "loss_fn":{
      "values":["smooth-loss", "cross-entropy"]
    },
    "lr_init":{
     "distribution":"uniform",
      "min": 0.0001,
      "max": 0.001
    },
    "lr_step":{
      "min": 10,
      "max": 100
    },
    "lr_gamma":{
      "distribution":"uniform",
      "min": 0.1,
      "max": 0.9
    },
    "max_rotation":{
      "distribution":"uniform",
      "min":0.0,
      "max":2*np.pi
    },
    "max_translation":{
      "distribution":"uniform",
      "min":0.0,
      "max":2.0
    },
    "min_scale":{
      "distribution":"uniform",
      "min":0.3,
      "max":1.0
    },
    "max_scale":{
      "distribution":"uniform",
      "min":1.0,
      "max":2.0
    }
  }
}

In [4]:
def interface_to_train():
    with wandb.init() as run:
        params={
        "dataset_type":type(trees_data),
        "batch_size":128,
        "validation_split":.15,
        "test_split":.15,
        "shuffle_dataset":True,
        "random_seed":0,
        "learning_rate":[wandb.config["lr_init"], wandb.config["lr_step"], wandb.config["lr_gamma"]],  #[init, step_size, gamma] for scheduler
        "momentum":0.9, #Only used for sgd
        "epochs":150,
        "loss_fn":wandb.config["loss_fn"],
        "optimizer":"adam",
        "voting":"None",
        "train_sampler":"balanced",

        "model":"SimpleView",

        "image_dim":256,
        "camera_fov_deg":90,
        "f":1,
        "camera_dist":1.4,
        "depth_averaging":"min",
        "soft_min_k":50,
        "num_views":6,

        "transforms":['rotation','translation','scaling'], #,'translation'
        "min_rotation":0,
        "max_rotation":wandb.config["max_rotation"],
        "min_translation":0,
        "max_translation":wandb.config["max_translation"],
        "jitter_std":0,
        "min_scale":wandb.config["min_scale"],
        "max_scale":wandb.config["max_scale"],

        "species":["QUEFAG", "PINNIG", "QUEILE", "PINSYL", "PINPIN"],
        "data_resolution":"2.5cm"
        }
        
        utils.train(data_dir=data_dir, model_dir=model_dir, params=params, init_wandb=False)
    return

In [ ]:
#sweep_id = wandb.sweep(sweep_config, project='laser-trees-bayes')
sweep_id = 'gf6k72md'

count = 3
wandb.agent(sweep_id, project='laser-trees-bayes', function=interface_to_train, count=count)

wandb: Agent Starting Run: royywwt5 with config:
wandb: 	loss_fn: smooth-loss
wandb: 	lr_gamma: 0.3075846456217122
wandb: 	lr_init: 0.0007674222266317178
wandb: 	lr_step: 12
wandb: 	max_rotation: 0.07941835325684538
wandb: 	max_scale: 1.1439212249822532
wandb: 	max_translation: 1.5068374006292433
wandb: 	min_scale: 0.842569376992901
wandb: Currently logged in as: mja2106 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Config item 'loss_fn' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_rotation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_translation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_scale' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_scale' was locked by 'sweep' (ignored update).


QUEFAG     1116
PINNIG      581
QUEILE      364
PINSYL      277
PINPIN      140
NA            2
JUNIPE        2
QUERCUS       2
DEAD          1
Name: sp, dtype: int64
Species:  ['DEAD', 'JUNIPE', 'NA', 'PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE', 'QUERCUS']
Labels:  tensor([8, 3, 6,  ..., 7, 3, 6])
Total count:  2485
Removing: QUERCUS
Removing: NA
Removing: DEAD
Removing: JUNIPE
Train Dataset:
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Validation Dataset (should match):
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Shuffling dataset...
Using balanced sampling...
Using smooth-loss
Optimizing with AdaM...
Using step LR scheduler...


/home/matt/anaconda3/envs/laser-trees/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,     5] loss: 4.009
[1,    10] loss: 3.808
[1,    15] loss: 3.792
[1,    20] loss: 3.810
OVERALL: Got 93 / 372 with accuracy 25.00
PINNIG: Got 93/93 with accuracy 100.00
PINPIN: Got 0/18 with accuracy 0.00
PINSYL: Got 0/40 with accuracy 0.00
QUEFAG: Got 0/177 with accuracy 0.00
QUEILE: Got 0/44 with accuracy 0.00
[2,     5] loss: 3.773
[2,    10] loss: 3.742
[2,    15] loss: 3.736
[2,    20] loss: 3.688
OVERALL: Got 57 / 372 with accuracy 15.32
PINNIG: Got 38/93 with accuracy 40.86
PINPIN: Got 14/18 with accuracy 77.78
PINSYL: Got 2/40 with accuracy 5.00
QUEFAG: Got 0/177 with accuracy 0.00
QUEILE: Got 3/44 with accuracy 6.82
[3,     5] loss: 3.685
[3,    10] loss: 3.742
[3,    15] loss: 3.656
[3,    20] loss: 3.613
OVERALL: Got 53 / 372 with accuracy 14.25
PINNIG: Got 0/93 with accuracy 0.00
PINPIN: Got 5/18 with accuracy 27.78
PINSYL: Got 24/40 with accuracy 60.00
QUEFAG: Got 0/177 with accuracy 0.00
QUEILE: Got 24/44 with accuracy 54.55
[4,     5] loss: 3.602
[4,    10] loss: 3.5

...

wandb: Network error resolved after 0:34:19.296703, resuming normal operation.


wandb: Network error resolved after 0:35:27.539501, resuming normal operation.


PINNIG Accuracy,0.72043
PINPIN Accuracy,0.61111
PINSYL Accuracy,0.75
QUEFAG Accuracy,0.87006
QUEILE Accuracy,0.72727
Best_acc,0.80108
Best_min_acc,0.7
Train Loss,2.28256
Validation Loss,0.63806
Train Accuracy,0.87127
Validation Accuracy,0.79032


PINNIG Accuracy,▆▁▂▄▄▁▅▄█▆▅▄▆▆██▇▄▇▆▆▆▆▇▆▇▇█▇████▇█▇██▇▇
PINPIN Accuracy,▁▁▄▅▁█▄▄▇▁▅▆▇▇▄▅▅█▆█▄▅▅▃▇▇▅▆▄▆▄▅▆▅▅▆▆▅▆▆
PINSYL Accuracy,▆▁▅▃█▅█▄▂█▄▄▅▅▆▅▃▇▆▄▇▆█▇▆▇▆▆▆▆▆▇▅▇▆▆▇▅▆▇
QUEFAG Accuracy,▁██▅▃▇▁▇▄▃▆▇▇▆▅▆█▆▆▆▇█▇▆▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇
QUEILE Accuracy,▂▁▁▇▇▂▅▆█▆▇▆▆▇▆▇▅▇▇▇▆▅▆▇▇▇▆▇▆▇▆▇▆▆▇▇▇▇▇▇
Best_acc,▁▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████████
Best_min_acc,▁▁▂▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████
Train Loss,▂█▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,▂█▁▁▂▂▃▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▁▃▄▄▄▃▅▅▄▆▆▇▇▆▇▆▆▇▆▇▇▇▇▇▇█▇▇███████████
Validation Accuracy,▁▄▅▅▄▅▂▆▅▄▆▆▇▆▆▇▇▆▇▆▇▇▇▇▇▇████████▇█████


wandb: Agent Starting Run: 4rmapbpd with config:
wandb: 	loss_fn: cross-entropy
wandb: 	lr_gamma: 0.267597245581744
wandb: 	lr_init: 0.0006404252141658939
wandb: 	lr_step: 100
wandb: 	max_rotation: 5.6833992393526
wandb: 	max_scale: 1.5141216239008035
wandb: 	max_translation: 0.03272868998694389
wandb: 	min_scale: 0.6272299976069986
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
